In [17]:
data = "/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/datasetok.csv"
text = "/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/text_modality/text_en.csv"

#merge two csv files
import pandas as pd
df1 = pd.read_csv(data)
df2 = pd.read_csv(text)
df1 = df1.drop(columns=['text'])
# df2 = df2.drop(columns=['label'])
# #merge two dataframes
# df1["text"] = df2["text"]

In [18]:
df2.head()

,text,label
0,Intracerebral hematoma cerebral contusion righ...,2
1,left subperitoneal hematoma left impaction lef...,1
2,Hematoma under the membrane_chronic_sale_bridge 2,1
3,brain contusion right hypochondrium right hypo...,1
4,Hematoma of the submental membrane of the fore...,2


In [19]:
#add thêm column record_ids từ file /home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/record_ids.csv vào data
record_ids = pd.read_csv("/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/record_ids.csv")
df1["record_id"] = record_ids["record_id"]
df1.head()

,age_at_record,sex,tbi_cli_reason,tbi_cli_time_acci_hos,tbi_cli_pulse,tbi_cli_temp,tbi_cli_blood_pressure,tbi_cli_breathing_rate,tbi_cli_glasgow,tbi_cli_awaken,...,albumin_v2,ure_v2,creatinin_v2,prothrombin_v2,d_3_aptt,d_4_dtim,d_target,tbi_ct_brain_parenchyma___7,tbi_ct_brain_parenchyma___8,record_id
0,52.0,0,1.0,2.0,70,36.5,2,19,13,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2,1.0,0,109.0
1,75.0,0,2.0,NaN,77,36.8,2,22,13,NaN,...,1.0,0.0,0.0,0.0,0.0,NaN,2,1.0,0,110.0
2,37.0,0,3.0,NaN,67,36.6,2,18,15,1.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2,0.0,0,112.0
3,66.0,0,4.0,NaN,75,36.5,2,19,14,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,3,0.0,0,113.0
4,20.0,0,2.0,4.0,85,36.8,2,18,15,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,2,0.0,0,114.0


In [20]:
#add thêm record_id từ file /home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/filtered_dataset.csv vào df2
df = pd.read_csv("/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/filtered_dataset.csv")
df2["record_id"] = df["record_id"]
df2.head()
#save df1 to /home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/merged_dataset.csv

,text,label,record_id
0,Intracerebral hematoma cerebral contusion righ...,2,72
1,left subperitoneal hematoma left impaction lef...,1,109
2,Hematoma under the membrane_chronic_sale_bridge 2,1,110
3,brain contusion right hypochondrium right hypo...,1,112
4,Hematoma of the submental membrane of the fore...,2,113


In [22]:
#match df2["text"] vào df1["text"] theo record_id
df1['text'] = df1['record_id'].map(df2.set_index('record_id')['text'])


In [25]:
df1.head()
df1.to_csv("/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/dataset/raw/merged_dataset.csv", index=False)
#check missing data in df1["text"]
missing_text = df1[df1['text'].isnull()]
print("Missing text records:", len(missing_text))
#check missing data in df2["text"]

Missing text records: 5


In [12]:
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm

# Danh sách các mô hình y tế
model_names = [
    "emilyalsentzer/Bio_ClinicalBERT",
    "dmis-lab/biobert-base-cased-v1.1",
    "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12",
    "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
    "yikuan8/Clinical-Longformer"
]

# Lặp qua từng mô hình
for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    max_len = 0
    lengths = []

    for i in tqdm(range(len(df1)), desc=f"Tokenizing with {model_name}"):
        text = df1["text"][i]
        if isinstance(text, str):
            tokens = tokenizer.tokenize(text)
            token_count = len(tokens)
            lengths.append(token_count)
            if token_count > max_len:
                max_len = token_count

    print(f"\n📌 Model: {model_name}")
    print(f"  → Max token length: {max_len}")
    print(f"  → 90th percentile: {pd.Series(lengths).quantile(0.9):.2f}")
    print(f"  → Mean token length: {pd.Series(lengths).mean():.2f}")


Tokenizing with emilyalsentzer/Bio_ClinicalBERT: 100%|██████████| 503/503 [00:00<00:00, 3607.25it/s]



📌 Model: emilyalsentzer/Bio_ClinicalBERT
  → Max token length: 803
  → 90th percentile: 90.00
  → Mean token length: 55.82


Tokenizing with dmis-lab/biobert-base-cased-v1.1: 100%|██████████| 503/503 [00:00<00:00, 3474.42it/s]



📌 Model: dmis-lab/biobert-base-cased-v1.1
  → Max token length: 803
  → 90th percentile: 90.00
  → Mean token length: 55.82


Tokenizing with bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12: 100%|██████████| 503/503 [00:00<00:00, 3664.22it/s]



📌 Model: bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12
  → Max token length: 803
  → 90th percentile: 89.00
  → Mean token length: 54.17


Tokenizing with microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract: 100%|██████████| 503/503 [00:00<00:00, 4083.75it/s]



📌 Model: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract
  → Max token length: 450
  → 90th percentile: 58.00
  → Mean token length: 38.39


Tokenizing with yikuan8/Clinical-Longformer: 100%|██████████| 503/503 [00:00<00:00, 4231.00it/s]


📌 Model: yikuan8/Clinical-Longformer
  → Max token length: 638
  → 90th percentile: 79.00
  → Mean token length: 51.31


In [23]:
#in ra max len của df1["text"]
max_len = 0
for i in range(len(df1)):
    if len(df1["text"][i]) > max_len:
        max_len = len(df1["text"][i])
print(max_len)

TypeError: object of type 'float' has no len()